In [424]:
# import libraries
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns 
import html
from bs4 import BeautifulSoup
import requests

In [425]:
my_page = requests.get(
    "https://www.railpassengers.org/all-aboard/tools-info/ridership-statistics/")
my_page.status_code

200

In [426]:
star_soup = BeautifulSoup(my_page.content, 'html.parser')

In [427]:
station_list = []
for l in star_soup.findAll('option'):
    station_list.append(l.contents)

station_value = []
for l in star_soup.findAll('option'):
    station_value.append(l['value'])

In [428]:
import pandas as pd
# Create a zip object from two lists
zipbObj = zip(station_list, station_value)
# Create a dictionary from zip object

labels = ['report type', 'url']
df = pd.DataFrame.from_records(zipbObj, columns=labels)
df['url']="https://www.railpassengers.org/"+df['url']

df.head()

,report type,url
0,[Select a State],https://www.railpassengers.org/
1,[ Alabama ],https://www.railpassengers.org//site/assets/fi...
2,[ Arizona ],https://www.railpassengers.org//site/assets/fi...
3,[ Arkansas ],https://www.railpassengers.org//site/assets/fi...
4,[ California ...,https://www.railpassengers.org//site/assets/fi...


In [429]:
df.index.values

array([   0,    1,    2, ..., 1081, 1082, 1083])

In [430]:
df['row_num'] = np.arange(len(df))
df.head()

,report type,url,row_num
0,[Select a State],https://www.railpassengers.org/,0
1,[ Alabama ],https://www.railpassengers.org//site/assets/fi...,1
2,[ Arizona ],https://www.railpassengers.org//site/assets/fi...,2
3,[ Arkansas ],https://www.railpassengers.org//site/assets/fi...,3
4,[ California ...,https://www.railpassengers.org//site/assets/fi...,4


In [431]:
#rows 2-49 are states
#rows 51-594 are stations
#rows 596-599 are business lines
#rows 601-647 are routes
#rows 649-1085 are congressional districts
#wip to fix here...
x=1
df['type']="state"
df.loc[df['row_num'] > 49, 'type'] = "stations"
df.loc[df['row_num'] >595, 'type'] = "business lines"
df.loc[df['row_num'] > 600, 'type'] = "routes"
df.loc[df['row_num'] >648, 'type'] = "congressional districts"

df2 = df
df2 = df2.drop(df2.index[[0, 49, 594, 599, 647]])
df2.head()

,report type,url,row_num,type
1,[ Alabama ],https://www.railpassengers.org//site/assets/fi...,1,state
2,[ Arizona ],https://www.railpassengers.org//site/assets/fi...,2,state
3,[ Arkansas ],https://www.railpassengers.org//site/assets/fi...,3,state
4,[ California ...,https://www.railpassengers.org//site/assets/fi...,4,state
5,[ Colorado ],https://www.railpassengers.org//site/assets/fi...,5,state


In [11]:
states = df2.loc[df2['type']=='state']
len(states)

48

In [175]:
state_list=[]

for i in range(0, len(states)):
#    print(states.iloc[i]['url'], states.iloc[i]['report type'])
    x=str(states.iloc[i]['report type'])
    x=x[24:]
    x=x[:-20]
    state_list.append(x)


In [177]:
states["states"]=state_list
states.head()

,report type,url,row_num,type,states
1,[ Alabama ],https://www.railpassengers.org//site/assets/fi...,1,state,Alabama
2,[ Arizona ],https://www.railpassengers.org//site/assets/fi...,2,state,Arizona
3,[ Arkansas ],https://www.railpassengers.org//site/assets/fi...,3,state,Arkansas
4,[ California ...,https://www.railpassengers.org//site/assets/fi...,4,state,California
5,[ Colorado ],https://www.railpassengers.org//site/assets/fi...,5,state,Colorado


In [458]:
#Download all pdfs
from urllib.request import Request, urlopen 
import requests  
import urllib.request  


#url="https://www.railpassengers.org/site/assets/files/1679/abe.pdf"  
req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})  

for i in range(0, len(states)):
#    print(states.iloc[i]['url'], states.iloc[i]['report type'])
    url=states.iloc[i]['url']
    r = requests.get(url)
    file_name="/Users/ericenglin/Desktop/Data Science/Amtrak Web Scraper/states/"
    file_name=file_name+(states.iloc[i]['states'])+".pdf"
    with open(file_name, "wb") as code:
        code.write(r.content)

In [421]:
## Read pdfs
from tabula import read_pdf

state_table=pd.DataFrame()
years = ['2012','2013','2014','2015','2016','2017','2018']



for i in range(0, len(states)):
    state=states.iloc[i]['states']
    print(state) #check status during loop
    #need to define file directory
    file_name="./../"+(states.iloc[i]['states'])+".pdf"
    #read pdf as new variable using tabula 
    df=read_pdf(file_name, 
                    spreadsheet=True, 
                    area=(101.118,29.647,291.706,342.0))
    vals=[]
    #header becomes the 2012 value when reading in table
    try:
        x_list=df.columns.values.tolist()
        for j in x_list:
            x=j.replace(",","")

        x=float(x)
        vals.append(x)

        value_list = df.values.tolist()
        for y in value_list:
            z=y[0]
            z=str(z)
            z=z.replace(",","")
            z=float(z)
            vals.append(z)

        new_state=pd.DataFrame()
        new_state['Year'] = years
        print(vals)


        new_state['Ridership']=vals
        new_state["state"]=state
        state_table = pd.concat([state_table, new_state])

    except AttributeError:
        print("No Data")



print(state_table)

Alabama
[66403.0, 65213.0, 62426.0, 60167.0, 57398.0, 56382.0, 53831.0]
Arizona
[104.9, 105.3, 104.4, 109.6, 107.3, 109.7, 104.1]
Arkansas
[40899.0, 40641.0, 36470.0, 34800.0, 32196.0, 34807.0, 33548.0]
California
[963.0, 1196.0, 527.0, 951.0, 1229.0, 1338.0, 1665.0]
Colorado
[202.2, 193.8, 199.3, 223.3, 246.0, 273.9, 263.2]
Connecticut
[1757.6, 1730.9, 1693.9, 1665.9, 1566.3, 1546.5, 1666.8]
Delaware
[741.8, 740.4, 707.8, 707.1, 697.0, 693.4, 710.3]
Florida
[1140.1, 1099.3, 1067.5, 1016.9, 936.4, 909.2, 885.0]
Georgia
[197.2, 187.5, 175.3, 162.5, 151.2, 150.9, 141.5]
Idaho
[8815.0, 9596.0, 8489.0, 7867.0, 7510.0, 7366.0, 7166.0]
Illinois
[4962.8, 5063.6, 4884.4, 4753.3, 4580.0, 4738.7, 4679.3]
Indiana
[148.5, 152.4, 147.4, 133.6, 132.5, 138.1, 130.6]
Iowa
[59695.0, 59035.0, 56599.0, 57508.0, 61247.0, 60434.0, 57999.0]
Kansas
[49498.0, 50146.0, 49418.0, 49673.0, 49484.0, 52214.0, 49476.0]
Kentucky
[10781.0, 11116.0, 10067.0, 10055.0, 9600.0, 9290.0, 8495.0]
Louisiana
[253.1, 243.9, 232

In [ ]:
state_table.to_csv("State Table.csv")